In [1]:
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import logging
import loger


import itertools
import pymysql
import re
import time
import random

In [22]:
# 데이터베이스 연결
sql_connect = pymysql.Connect(
    host = '127.0.0.1',#'129.154.62.175',
    user = 'root',#'streamlit',
    password = '1234',#'bigdata1!',
    db = 'streamlit_db',#db 이름 입력,
    charset = 'utf8mb4'
)

# cursor 생성(통로 역할)
cursor = sql_connect.cursor()

# your_gg 테이블에 데이터 조회 쿼리문 전송
cursor.execute('select * from op_gg')

# fetchall함수는 테이블에있는 모든 데이터를 가지고옴 
load_data = cursor.fetchall()

# load_data의 id열은 제외한 user_id만 데이터프레임 형태로 저장
user_name = pd.DataFrame(load_data)[1]

In [23]:
# 크롤링한 데이터 your_gg 테이블 전체를 불러와서 데이터 넣어줌
slq = 'insert into your_gg values(%s)'

In [24]:
#logging 결과 텍스트 파일 위치
log_file_path = 'E:/토이프로젝트/log/name_crawling_log.log'
logg = loger.mylog(log_file_path)

#크롬 driver 위치
driver_path = 'E:/토이프로젝트/crawling_data/chromedriver'

In [29]:
driver = webdriver.Chrome(executable_path='driver_path')

for x,i in enumerate(user_name):
    # 닉네임 가져와서 URL로 접근
    URL = 'https://your.gg/ko/kr/profile/{}'.format(i)
    driver.get(url=URL)
    # 10초 기다리는데 그 전에 페이지 로딩되면 다음 코드로 넘어감
    driver.implicitly_wait(10)
    # op.gg에서 수집한 닉네임을 your.gg에서 크롤링할 당시 닉네임 변경한 사람이 있을까봐 예외처리함
    try:
        # 전적갱신 버튼 눌러줌
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/header/div/div[2]/button').click()
        time.sleep(3)
        driver.implicitly_wait(10) # 4~8초 사이 랜덤으로 타임슬립 걸어줌 <-- 이거 고쳤습니다

        # 지표 더 보기 버튼 클릭
        driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[1]/div/div[2]/div[2]/button').click()
        driver.implicitly_wait(2)
        
        # target에는 현재 시즌 티어가 저장됨
        target = driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/header/div/div[1]/div[2]/div/span[1]').text
        
        # 전 시즌 티어 저장시 작년에 롤을 하지 않은 사람도 있기 때문에 예외처리
        try:
            before_season = driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/header/div/div[1]/div[3]/div[1]').text.split('\n')[1]
        except:
            before_season = 'unranked'
        
        # 주 포지션 저장
        main_position = driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[1]/div/div[2]/div[2]/div[1]/div[2]/div[1]/label[1]/div/span[1]').text 
        
        # 승,패 합친 판 수 데이터를 전처리하여 num_game 변수에 저장
        num_gamee = driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/header/div/div[1]/div[2]/div/span[3]').text
        num_game = int(re.sub('\D','',num_gamee.split()[2])) + int(re.sub('\D','',num_gamee.split()[3]))
        
        # 지표 더 보기 통해 나온 22개의 데이터들을 한꺼번에 들고와서 그 값만 리스트로 저장
        info = driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div[1]/div/div[2]/div[2]/div[3]/div/ul').text.split('\n')
        logg.log('유저: {} 크롤링'.format(i), level = logging.warning)
        result = [info[i * 3:(i + 1) * 3][0] for i in range((len(info) + 3 - 1) // 3 )]
        print([info[i * 3:(i + 1) * 3][0] for i in range((len(info) + 3 - 1) // 3 )])
        
        # 닉네임, 전시즌티어, 주포지션, 총판수, 22개 데이터, 현시즌티어 데이터 저장 
        va = tuple(list([i,before_season,main_position,num_game] + result + [target]))
        print(va)
        
        # # SQL에 데이터 입력 쿼리 던짐
        # cursor.execute(slq, va)
        
        # # 입력한 데이터 저장
        # sql_connect.commit()
        # logg.log('유저: {} MySQL Insert'.format(i), level = logging.warning)
    except:
        pass

# 연결자 종료
# sql_connect.close()
driver.quit()

C:\Users\USER\AppData\Local\Temp\ipykernel_27480\4026347286.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='driver_path')
